<a href="https://colab.research.google.com/github/BeckMarquez/Ukrenergo/blob/main/NBU_rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request, json
import pandas as pd

In [ ]:
# Параметри запиту
# Дата початку періоду у форматі YYYYMMDD без роздільників
start='20240601'

# Дата завершення періоду у форматі YYYYMMDD без роздільників
end='20240630'

# Смвольний код валюти (не обов’язковий параметр), якщо цей параметр не заначено, то у результаті буде вибірка за період по всім валютам
valcode='eur'

# Назва поля по якому виконується сортування (exchangedate/r030/сс/rate)
sort='exchangedate'

# Метод сортування (desc – за спаданням, asc – за зростанням)
order='asc'

# Формат надання інформації, якщо цей параметр не зазначено, то XML
format='json'

In [ ]:
# Приклад URl
# url="https://bank.gov.ua/NBU_Exchange/exchange_site?start=yyyymmdd&end=%20yyyymmdd&valcode=usd&sort=exchangedate&order=desc&json"

# Формування URL на основі параметрів запиту
url='https://bank.gov.ua/NBU_Exchange/exchange_site?start={0}&end=%20{1}&valcode={2}&sort={3}&order={4}&{5}'.format(start, end, valcode, sort, order, format)

In [ ]:
with urllib.request.urlopen(url) as url:
    data = json.load(url)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df = df.drop(['txt', 'enname', 'units', 'rate_per_unit', 'group'], axis=1)
df['r030'] = df['r030'].astype(str)
df.set_index('exchangedate', inplace=True)

In [ ]:
df = pd.concat([df, df.mean(numeric_only=True).to_frame('Середній курс ЄВРО - гривня').T]).fillna('')
df = pd.concat([df, 1/df.mean(numeric_only=True).to_frame('Середній курс гривня - ЄВРО').T]).fillna('')

In [ ]:
df.to_excel('output_{0}_{1}_{2}.xlsx'.format(start, end, valcode), index_label='Дата', header=['Код валюти', 'Назва валюти', 'Курс', 'Дата встановлення'])